In [1]:
import json
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
import nltk
import string
import re 
from re import search
nltk.download('stopwords')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
snow_stemmer = nltk.SnowballStemmer('english')
nltk.download('wordnet')
from nltk.corpus import stopwords
import spacy
#nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/willcline/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/willcline/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pickle
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
with open('encoder.pkl','rb') as f:
    encoder = pickle.load(f)

/Users/willcline/opt/anaconda3/envs/learn-env/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
string_='After more than seven months of rotating guest hosts, the search for the new host of “Jeopardy!” came to an anticlimactic end on Wednesday when Sony Pictures Television announced executive producer Mike Richards as the show’s regular host starting this fall.“Jeopardy!” is also adding a second host for the first time in what feels like a concession to the widespread criticism of Richards when he was rumored to be the front-runner to take over for the late Alex Trebek. Actor Mayim Bialik will host prime-time specials and spinoffs, including an upcoming prime-time version of the show’s longtime college tournament.'

In [4]:
string_ = 'It doesn’t have to be cold outside for you to catch a cold. Although it’s more common to get sick in the winter, viruses can circulate year-round. Close proximity to others, lax hygiene, stress, lack of sleep and more can contribute to your likelihood of contracting an illness ― even in hotter weather. Colds are usually rare in the summer, and are often caused by viruses that are different from those we see in the winter. Now that people are meeting up and seeing each other more, these viruses are starting to circulate again. That means we’re seeing many more “cold viruses” this summer than usual, according to Ellen F. Foxman, assistant professor in Yale School of Medicine’s Department of Laboratory Medicine.And as anyone who has had a summer cold knows, the symptoms can feel even worse during this time of year. (Shoutout to everyone who has caught a cold lately while living their post-COVID vaccine life).'

In [5]:
string_

'It doesn’t have to be cold outside for you to catch a cold. Although it’s more common to get sick in the winter, viruses can circulate year-round. Close proximity to others, lax hygiene, stress, lack of sleep and more can contribute to your likelihood of contracting an illness ― even in hotter weather. Colds are usually rare in the summer, and are often caused by viruses that are different from those we see in the winter. Now that people are meeting up and seeing each other more, these viruses are starting to circulate again. That means we’re seeing many more “cold viruses” this summer than usual, according to Ellen F. Foxman, assistant professor in Yale School of Medicine’s Department of Laboratory Medicine.And as anyone who has had a summer cold knows, the symptoms can feel even worse during this time of year. (Shoutout to everyone who has caught a cold lately while living their post-COVID vaccine life).'

In [6]:
from tqdm import tqdm

stop = stopwords.words('english')
def clean_history(text):
    text = re.sub('<a\b[^>]*>(.*?)</a>', 
       '',text)
    punct_translator=str.maketrans('','',string.punctuation.replace('.', '') + '―“”’')
    digit_translator=str.maketrans('','',string.digits)
    text=text.translate(punct_translator)
    text=text.translate(digit_translator)
    split = text.split()
    text = " ".join([wn.lemmatize(word.lower()) for word in split if word.lower() not in stop and '.com' not in word and 'pictwitter' not in word and 'http' not in word and 'www' not in word])
#    
    return text
string_=clean_history(string_)



In [7]:
with open('tokenizer.pkl','rb') as f:
    tokenizer = pickle.load(f)

In [8]:
text_seq = tokenizer.texts_to_sequences([string_])

In [9]:
text_seq = pad_sequences(text_seq, maxlen=1000, truncating='post')

In [10]:
text_seq.shape

(1, 1000)

In [11]:
from tensorflow.keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [12]:
#text_seq.reshape(1, 1000, 448).shape

In [23]:
x = np.expand_dims(text_seq, 0)

In [21]:
prediction = loaded_model.predict_classes(text_seq)

In [22]:
prediction = encoder.inverse_transform(prediction)

In [39]:
prediction.tostring()

AttributeError: 'numpy.ndarray' object has no attribute 'to_string'

In [49]:
str(prediction)[2:-2].lower().title()

'Healthy Living'

In [37]:
type(prediction)

numpy.ndarray